In [1]:
# Import required modules
import pandas as pd
from concurrent.futures import ProcessPoolExecutor
import numpy as np

In [2]:
# Read in the data to analyse. Unzip "combinedOrdersdata.zip" file
df = pd.read_pickle("combined_orders_data.pickle")

In [4]:
# Create a class to analyse data
class Analyse:
    
    # Calculates distribution of a variable across another variable
    def calIndDist(groupBy, filterBy, toGroup):
        """groupBy = The variable used to group another variable,
        filterBy = any class of 'groupBy' variable,
        toGroup = The variable that will be grouped"""
        
        # Distribution in absolute number of a variable across another variable
        indCount = df[df[f"{groupBy}"]==filterBy][f"{toGroup}"].value_counts().to_frame()\
        .T.reset_index(drop=True)
        indCount[f"{groupBy}"] = filterBy
        
        # Distribution in percentage of a variable across another variable
        indPercentDist = df[df[f"{groupBy}"]==filterBy][f"{toGroup}"].value_counts(normalize=True)\
        .mul(100).to_frame().T.reset_index(drop=True)
        indPercentDist.columns = indPercentDist.columns + "_%"
        
        # Merge the 2 dfs on column-wise
        mergeInd = pd.concat([indCount, indPercentDist], axis=1)
        return mergeInd
        
    
    # Calculates a variable overall distribution
    def calCombDist(groupBy):
        """groupBy = The variable whose distribution is calculated"""
        
        # # Distribution in absolute number
        combCount = df[f"{groupBy}"].value_counts().to_frame().T.reset_index(drop=True)
        combCount[f"{groupBy}"] = "overall"
        
        # Distribution in %
        combPercentDist = df[f"{groupBy}"].value_counts(normalize=True)\
        .mul(100).to_frame().T.reset_index(drop=True)
        combPercentDist.columns = combPercentDist.columns + "_%"
        
        # Merge the 2 dfs on column-wise
        mergeComb = pd.concat([combCount, combPercentDist], axis=1)
        return mergeComb

### What is brand-wise and combined male-female students ratio?
#### Takeaway: Overall almost 36% students is male and just over 64% students are female.  John academy has the most female students(72.7%), while TX has the closest male-female students ratio(46.6% male) off all brands.

In [5]:
genderRatioByBrand = pd.concat(list(map(
    Analyse.calIndDist,
    ["courseProvider"]*df.courseProvider.nunique(),
    df.courseProvider.unique(),
    ["gender"]*df.courseProvider.nunique()))).reset_index(drop=True).round(2)

genderRatioOverall = Analyse.calCombDist("gender").round(2)
display(genderRatioOverall, genderRatioByBrand)

,F,M,gender,F_%,M_%
0,54067,30303,overall,64.08,35.92


,F,M,courseProvider,F_%,M_%
0,18270,12731,one education,58.93,41.07
1,8303,3524,alpha academy,70.20,29.80
2,7051,3202,janets,68.77,31.23
3,4285,3751,training express,53.32,46.68
4,8061,3028,john academy,72.69,27.31
5,8097,4067,course gate,66.57,33.43


### What is brand-wise and combined payment method ratio?
#### Takeaway: Combining all brands, over 51% prefers debit or credit card, almost 43% prefers paypal and rest 6% prefers others. John acdemy students prefer only debit or credit card an paypal. Course gate student has the highest apple pay ratio compared to all brands.

In [6]:
paymentMethodByBrand = pd.concat(
list(map(Analyse.calIndDist,
        ["courseProvider"]*df.courseProvider.nunique(),
        df.courseProvider.unique(),
        ["paymentMethodTitle"]*df.courseProvider.nunique()))).reset_index(drop=True).fillna(0)

paymentMethodOverall = Analyse.calCombDist("paymentMethodTitle")
display(paymentMethodOverall, paymentMethodByBrand)

,debit or credit card,paypal,chrome payment request (stripe),apple pay (stripe),cash on delivery,paymentMethodTitle,debit or credit card_%,paypal_%,chrome payment request (stripe)_%,apple pay (stripe)_%,cash on delivery_%
0,41769,35094,2763,2044,3,overall,51.141748,42.968913,3.383003,2.502663,0.003673


,debit or credit card,paypal,chrome payment request (stripe),apple pay (stripe),cash on delivery,courseProvider,debit or credit card_%,paypal_%,chrome payment request (stripe)_%,apple pay (stripe)_%,cash on delivery_%
0,14686,13467,1108,820,3,one education,48.816647,44.764659,3.683021,2.725701,0.009972
1,5845,5045,481,122,0,alpha academy,50.857043,43.896285,4.185156,1.061516,0.0
2,5142,3879,399,388,0,janets,52.426591,39.549347,4.068108,3.955954,0.0
3,4205,2661,256,183,0,training express,57.563313,36.427105,3.504449,2.505133,0.0
4,6113,4844,0,0,0,john academy,55.790819,44.209181,0.0,0.0,0.0
5,5778,5198,519,531,0,course gate,48.045901,43.223017,4.315649,4.415433,0.0


### Which country cash on delivery was made from?
#### Takeaway: All the cash on delivery was made Bangladesh

In [7]:
df[df.paymentMethodTitle=="cash on delivery"].countryCodeBilling.unique()

<StringArray>
['bd']
Length: 1, dtype: string

### What is gender-wise payment method?
#### Takeaway: Male students tend to use debit or credit card more than female(52.3%/50.5%), while female student tend to use paypal more than male(38.9%/45%).  Over 6% male student use chrome payment requests, while it is only 1.8% for female.

In [8]:
paymentMethodByGender = pd.concat(
list(map(Analyse.calIndDist,
         ["gender"]*df.paymentMethodTitle.nunique(),
        df.gender.unique(),
        ["paymentMethodTitle"]*df.paymentMethodTitle.nunique()))).reset_index(drop=True).fillna(0)
display(paymentMethodByGender)

,debit or credit card,paypal,chrome payment request (stripe),apple pay (stripe),cash on delivery,gender,debit or credit card_%,paypal_%,chrome payment request (stripe)_%,apple pay (stripe)_%,cash on delivery_%
0,14792,11005,1797,681,3,M,52.309216,38.917179,6.354763,2.408233,0.010609
1,26977,24089,966,1363,0,F,50.523457,45.114711,1.809158,2.552673,0.0


### What is brand-wise students ratio by a country and top 10 countries by student?
#### Takeaway: Over 40.5% GB's students are from One Education, while it is only 2% for TX. Over 46% US's students are from John academy, while it is only 0% for TX. For Saudi Arabia(54,7%), Bahrain(41.5), Qatar(57.4), and UAE(52.4), its Alpha Academy that has the most students by a brand. For T&T, its Janets(31.8) who got the most share by a brand.

#### Undoubtedly, It GB from where our most of the students are (88%). Almost 1% students are from UAE, over 0.8% are from South Africa, 0.78% are from Saudi Arabia, and 0.74% are from Ireland.

In [9]:
studentByCountry = pd.concat(
list(map(Analyse.calIndDist,
        ["countryCodeBilling"]*df.countryCodeBilling.nunique(),
        df.countryCodeBilling.unique(),
        ["courseProvider"]*df.countryCodeBilling.nunique()))).dropna(how="all").fillna(0).reset_index(drop=True)


# Top 10 countries by student
studentCountByCountry = df.countryCodeBilling.value_counts().to_frame("count")\
.reset_index().rename(columns={"index":"country"}).head(10)
studentPercentByCountry = df.countryCodeBilling.value_counts(normalize=True).mul(100)\
.to_frame("percent").reset_index().rename(columns={"index":"country"}).head(10)

overAllStudentByCountry = pd.merge(studentCountByCountry, studentPercentByCountry, on="country")
display(studentByCountry.head(20), overAllStudentByCountry)

,one education,course gate,alpha academy,janets,john academy,training express,countryCodeBilling,one education_%,course gate_%,alpha academy_%,janets_%,john academy_%,training express_%
0,27877,11280,9584,9236,8986,1416,gb,40.768365,16.496293,14.015999,13.507071,13.141462,2.070811
1,124,60,55,34,236,0,us,24.361493,11.787819,10.805501,6.679764,46.365422,0.0
2,112,5,333,45,112,1,sa,18.421053,0.822368,54.769737,7.401316,18.421053,0.164474
3,15,1,27,0,22,0,bh,23.076923,1.538462,41.538462,0.0,33.846154,0.0
4,78,2,143,3,22,1,qa,31.325301,0.803213,57.429719,1.204819,8.835341,0.401606
5,138,19,393,26,170,3,ae,18.424566,2.536716,52.46996,3.471295,22.696929,0.400534
6,72,8,13,7,45,0,ca,49.655172,5.517241,8.965517,4.827586,31.034483,0.0
7,21,0,2,9,10,0,th,50.0,0.0,4.761905,21.428571,23.809524,0.0
8,233,66,167,45,61,3,ie,40.521739,11.478261,29.043478,7.826087,10.608696,0.521739
9,10,6,7,16,5,0,se,22.727273,13.636364,15.909091,36.363636,11.363636,0.0


,country,count,percent
0,gb,68379,87.947267
1,bd,1220,1.569132
2,ae,749,0.963344
3,za,649,0.834727
4,sa,608,0.781994
5,ie,575,0.73955
6,us,509,0.654662
7,ng,375,0.482315
8,qa,249,0.320257
9,es,211,0.271383


### What is country-wise male-female ratio?
#### Takeaway: For GB and US, Qatar, UAE and Ireland female students ratio outnumbers corresponding male students ratio. For Saudi Arabia, Bahrain, and Canada, male student ratio is higher than those female students ratio. The ratio is exaxctly 50%-50% for Sweden.

In [10]:
genderByCountry = pd.concat(
list(map(Analyse.calIndDist,
        ["countryCodeBilling"]*df.countryCodeBilling.nunique(),
        df.countryCodeBilling.unique(),
        ["gender"]*df.countryCodeBilling.nunique()))).dropna(how="all").fillna(0).reset_index(drop=True)
display(genderByCountry.head(10))

,F,M,countryCodeBilling,F_%,M_%
0,45802.0,22577.0,gb,66.982553,33.017447
1,307.0,202.0,us,60.314342,39.685658
2,250.0,358.0,sa,41.118421,58.881579
3,32.0,33.0,bh,49.230769,50.769231
4,137.0,112.0,qa,55.020080,44.979920
5,431.0,318.0,ae,57.543391,42.456609
6,53.0,92.0,ca,36.551724,63.448276
7,25.0,17.0,th,59.523810,40.476190
8,350.0,225.0,ie,60.869565,39.130435
9,22.0,22.0,se,50.000000,50.000000


### What is country-wise payment method ratio?
#### Takeaway: For most of the countries, debit or credit card is the favourite payment method system. Students from countries like Netherlands, Denmark, and China prefer paypal as their favourite payment method compared to other payment system.

In [11]:
paymentMethodByCountry = pd.concat(
list(map(Analyse.calIndDist,
        ["countryCodeBilling"]*df.countryCodeBilling.nunique(),
        df.countryCodeBilling.unique(),
        ["paymentMethodTitle"]*df.countryCodeBilling.nunique()))).dropna(how="all").fillna(0).reset_index(drop=True)
display(paymentMethodByCountry.head(20))

,debit or credit card,paypal,chrome payment request (stripe),apple pay (stripe),countryCodeBilling,debit or credit card_%,paypal_%,chrome payment request (stripe)_%,apple pay (stripe)_%,cash on delivery,cash on delivery_%
0,32985,30601,2394,1872,gb,48.613158,45.099629,3.528267,2.758946,0,0.0
1,252,192,10,0,us,55.506608,42.290749,2.202643,0.0,0,0.0
2,518,39,19,14,sa,87.79661,6.610169,3.220339,2.372881,0,0.0
3,52,10,1,0,bh,82.539683,15.873016,1.587302,0.0,0,0.0
4,166,65,0,0,qa,71.861472,28.138528,0.0,0.0,0,0.0
5,651,54,5,16,ae,89.669421,7.438017,0.688705,2.203857,0,0.0
6,100,37,0,0,ca,72.992701,27.007299,0.0,0.0,0,0.0
7,32,10,0,0,th,76.190476,23.809524,0.0,0.0,0,0.0
8,297,239,31,3,ie,52.105263,41.929825,5.438596,0.526316,0,0.0
9,29,12,3,0,se,65.909091,27.272727,6.818182,0.0,0,0.0


### What is country-wise and overall order status?
#### Takeaway: Overall over 99% order status are complete, almost 0.5% is pending, 0.41% is in processing and 0.046% is on hold.
#### Bahrain, Sweden, Thailand and Canada have no other status except completed. Ireland has the most (0.87%) on hold order status followed by US.

In [12]:
orderStatusByCountry = pd.concat(
list(map(Analyse.calIndDist,
        ["countryCodeBilling"]*df.countryCodeBilling.nunique(),
        df.countryCodeBilling.unique(),
        ["orderStatus"]*df.countryCodeBilling.nunique()))).dropna(how="all").fillna(0).reset_index(drop=True)
orderStatusOverall = Analyse.calCombDist("orderStatus")
display(orderStatusOverall, orderStatusByCountry.head(10))

,completed,pending payment,processing,on hold,orderStatus,completed_%,pending payment_%,processing_%,on hold_%
0,83569,413,349,39,overall,99.05061,0.48951,0.413654,0.046225


,completed,pending payment,processing,on hold,countryCodeBilling,completed_%,pending payment_%,processing_%,on hold_%
0,67660,370,319,30,gb,98.948508,0.541102,0.466517,0.043873
1,505,0,2,2,us,99.214145,0.0,0.392927,0.392927
2,605,3,0,0,sa,99.506579,0.493421,0.0,0.0
3,65,0,0,0,bh,100.0,0.0,0.0,0.0
4,246,3,0,0,qa,98.795181,1.204819,0.0,0.0
5,746,3,0,0,ae,99.599466,0.400534,0.0,0.0
6,145,0,0,0,ca,100.0,0.0,0.0,0.0
7,42,0,0,0,th,100.0,0.0,0.0,0.0
8,570,0,0,5,ie,99.130435,0.0,0.0,0.869565
9,44,0,0,0,se,100.0,0.0,0.0,0.0


### What is brand-wise order status?
#### Takeaway: Janets has the most (3%) "processing" order status. One education has the most (1.12) "pending" order status. John and Alpha academy have all the orders completed.

In [13]:
orderStatusByBrand = pd.concat(
list(map(Analyse.calIndDist,
        ["courseProvider"]*df.courseProvider.nunique(),
        df.courseProvider.unique(),
        ["orderStatus"]*df.courseProvider.nunique()))).dropna(how="all").fillna(0).reset_index(drop=True)
display(orderStatusByBrand)

,completed,pending payment,processing,on hold,courseProvider,completed_%,pending payment_%,processing_%,on hold_%
0,30595,366,33,7,one education,98.690365,1.180607,0.106448,0.02258
1,11827,0,0,0,alpha academy,100.0,0.0,0.0,0.0
2,9881,47,316,9,janets,96.371794,0.458402,3.082025,0.087779
3,8034,0,0,2,training express,99.975112,0.0,0.0,0.024888
4,11089,0,0,0,john academy,100.0,0.0,0.0,0.0
5,12143,0,0,21,course gate,99.827359,0.0,0.0,0.172641


### What is gender-wise order status?
#### Takeaway: Female students have almost 90% order status "on hold" compared to male students.

In [14]:
orderStatusByGender = pd.concat(
list(map(Analyse.calIndDist,
        ["orderStatus"]*df.orderStatus.nunique(),
        df.orderStatus.unique(),
        ["gender"]*df.orderStatus.nunique()))).dropna(how="all").fillna(0).reset_index(drop=True)
display(orderStatusByGender)

,F,M,orderStatus,F_%,M_%
0,53539,30030,completed,64.065622,35.934378
1,261,152,pending payment,63.196126,36.803874
2,232,117,processing,66.475645,33.524355
3,35,4,on hold,89.743590,10.256410


### What is state-wise male-female ratio?
#### Takeaway: For every state, male students are outnumbered by female students except for Chesire, Newton mearns, and KSA.

In [15]:
genderByState = pd.concat(
list(map(Analyse.calIndDist,
        ["stateCodeBilling"]*df.stateCodeBilling.nunique(),
        df.stateCodeBilling.unique(),
        ["gender"]*df.stateCodeBilling.nunique()))).dropna(how="all").fillna(0).reset_index(drop=True)
display(genderByState.head(20))

,stateCodeBilling,F,M,F_%,M_%
0,kent,1245.0,464.0,72.849620,27.150380
1,highland,20.0,2.0,90.909091,9.090909
2,county (optional),149.0,80.0,65.065502,34.934498
3,england,1316.0,583.0,69.299631,30.700369
4,west sussex,370.0,115.0,76.288660,23.711340
5,co durham,31.0,6.0,83.783784,16.216216
6,leicestershire,288.0,140.0,67.289720,32.710280
7,chesire,0.0,2.0,0.000000,100.000000
8,staffordshire,332.0,155.0,68.172485,31.827515
9,wiltshire,236.0,110.0,68.208092,31.791908


### What is state-wise payment method?
#### Takeaway: England students mostly pay through apple pay than any other payment methods. Highland, West sussex, Cheshire, and Leicestershire students use paypal more than any other payment methods.

In [16]:
paymentMethodByState = pd.concat(
list(map(Analyse.calIndDist,
        ["stateCodeBilling"]*df.stateCodeBilling.nunique(),
        df.stateCodeBilling.unique(),
        ["paymentMethodTitle"]*df.stateCodeBilling.nunique()))).dropna(how="all").fillna(0).reset_index(drop=True)
display(paymentMethodByState.head(20))

,stateCodeBilling,debit or credit card,paypal,chrome payment request (stripe),apple pay (stripe),debit or credit card_%,paypal_%,chrome payment request (stripe)_%,apple pay (stripe)_%,cash on delivery,cash on delivery_%
0,kent,917,735,33,22,53.719977,43.057996,1.933216,1.288811,0,0.0
1,highland,4,15,3,0,18.181818,68.181818,13.636364,0.0,0,0.0
2,county (optional),149,69,11,0,65.065502,30.131004,4.803493,0.0,0,0.0
3,england,588,508,8,791,31.029024,26.807388,0.422164,41.741425,0,0.0
4,west sussex,219,247,10,6,45.435685,51.244813,2.074689,1.244813,0,0.0
5,co durham,12,16,0,9,32.432432,43.243243,0.0,24.324324,0,0.0
6,leicestershire,194,219,6,2,46.08076,52.019002,1.425178,0.475059,0,0.0
7,chesire,2,0,0,0,100.0,0.0,0.0,0.0,0,0.0
8,staffordshire,213,260,4,9,43.82716,53.497942,0.823045,1.851852,0,0.0
9,wiltshire,162,172,3,9,46.820809,49.710983,0.867052,2.601156,0,0.0


### what is state-wise order status?
#### Takeaway: Students from Kent has the highest pending payment ratio than any other state. Students from Staffordshire has the highest "on hold" order status. Students from Leicestershire has the highest "processing" order status. There is no order status except completed for states Highland, Co durham, and Chesire.

In [17]:
orderStatusByState = pd.concat(
list(map(Analyse.calIndDist,
        ["stateCodeBilling"]*df.stateCodeBilling.nunique(),
        df.stateCodeBilling.unique(),
        ["orderStatus"]*df.stateCodeBilling.nunique()))).dropna(how="all").fillna(0).reset_index(drop=True)
display(orderStatusByState.head(10))

,stateCodeBilling,completed,processing,pending payment,on hold,completed_%,processing_%,pending payment_%,on hold_%
0,kent,1672,18,17,2,97.834991,1.053248,0.994734,0.117028
1,highland,22,0,0,0,100.0,0.0,0.0,0.0
2,county (optional),229,0,0,0,100.0,0.0,0.0,0.0
3,england,1890,4,5,0,99.526066,0.210637,0.263296,0.0
4,west sussex,483,0,2,0,99.587629,0.0,0.412371,0.0
5,co durham,37,0,0,0,100.0,0.0,0.0,0.0
6,leicestershire,419,9,0,0,97.897196,2.102804,0.0,0.0
7,chesire,2,0,0,0,100.0,0.0,0.0,0.0
8,staffordshire,483,1,1,2,99.178645,0.205339,0.205339,0.410678
9,wiltshire,344,2,0,0,99.421965,0.578035,0.0,0.0


### What is state-wise course provider?
#### Takeaway: Kent, Highland, Englang, West sussex's most of the students are from One education. Most TX courses are sold in Northants while most John academy courses are sold in West sussesx.

In [18]:
courseProviderByState = pd.concat(
list(map(Analyse.calIndDist,
        ["stateCodeBilling"]*df.stateCodeBilling.nunique(),
        df.stateCodeBilling.unique(),
        ["courseProvider"]*df.stateCodeBilling.nunique()))).dropna(how="all").fillna(0).reset_index(drop=True)
display(courseProviderByState.head(15))

,stateCodeBilling,one education,janets,course gate,alpha academy,john academy,training express,one education_%,janets_%,course gate_%,alpha academy_%,john academy_%,training express_%
0,kent,716,268,254,246,202,23,41.895846,15.681685,14.862493,14.394383,11.819778,1.345816
1,highland,12,0,2,5,3,0,54.545455,0.0,9.090909,22.727273,13.636364,0.0
2,county (optional),88,38,51,23,24,5,38.427948,16.593886,22.270742,10.043668,10.480349,2.183406
3,england,900,287,440,136,94,42,47.393365,15.113217,23.17009,7.161664,4.949974,2.21169
4,west sussex,155,43,61,107,116,3,31.958763,8.865979,12.57732,22.061856,23.917526,0.618557
5,co durham,19,13,0,4,0,1,51.351351,35.135135,0.0,10.810811,0.0,2.702703
6,leicestershire,176,63,64,60,57,8,41.121495,14.719626,14.953271,14.018692,13.317757,1.869159
7,chesire,2,0,0,0,0,0,100.0,0.0,0.0,0.0,0.0,0.0
8,staffordshire,204,45,67,100,58,13,41.889117,9.240246,13.7577,20.533881,11.909651,2.669405
9,wiltshire,154,49,46,54,40,3,44.508671,14.16185,13.294798,15.606936,11.560694,0.867052


### What is city-wise gender ratio?
#### Takeaway: Bishop auckland (m/f=60%/40%) and Northallerton (58%42%) have higher male than female students. For most of the city, female students ratio is higher than male.

In [19]:
%%time
genderByCity = pd.concat(
    list(map(Analyse.calIndDist,
            ["cityBilling"]*df.cityBilling.nunique(),
            df.cityBilling.unique(),
            ["gender"]*df.cityBilling.nunique()))).dropna(how="all").fillna(0).reset_index(drop=True)
display(genderByCity.head(15))

,F,M,cityBilling,F_%,M_%
0,32.0,16.0,bilston,66.666667,33.333333
1,84.0,29.0,rochester,74.336283,25.663717
2,600.0,374.0,manchester,61.601643,38.398357
3,21.0,6.0,inverness,77.777778,22.222222
4,0.0,1.0,ewloe,0.000000,100.000000
5,18.0,27.0,bishop auckland,40.000000,60.000000
6,0.0,5.0,"abram, wigan",0.000000,100.000000
7,51.0,35.0,wrexham,59.302326,40.697674
8,6.0,4.0,leven,60.000000,40.000000
9,45.0,6.0,bognor regis,88.235294,11.764706


CPU times: user 1min 4s, sys: 16 ms, total: 1min 4s
Wall time: 1min 4s


### What is provider ratio by city?
#### Takeaway: One education, as expected, is the best provider for most of the cities except Bognor regis where its Course gate with most of the students.

In [20]:
%%time
providerByCity = pd.concat(
    list(map(Analyse.calIndDist,
            ["cityBilling"]*df.cityBilling.nunique(),
            df.cityBilling.unique(),
            ["courseProvider"]*df.cityBilling.nunique()))).dropna(how="all").fillna(0).reset_index(drop=True)
display(providerByCity.head(15))

,one education,alpha academy,course gate,training express,janets,john academy,cityBilling,one education_%,alpha academy_%,course gate_%,training express_%,janets_%,john academy_%
0,22,9,8,5,2,2,bilston,45.833333,18.75,16.666667,10.416667,4.166667,4.166667
1,36,11,24,2,14,26,rochester,31.858407,9.734513,21.238938,1.769912,12.389381,23.00885
2,370,158,173,14,140,119,manchester,37.98768,16.221766,17.761807,1.437372,14.373717,12.217659
3,12,8,3,0,0,4,inverness,44.444444,29.62963,11.111111,0.0,0.0,14.814815
4,1,0,0,0,0,0,ewloe,100.0,0.0,0.0,0.0,0.0,0.0
5,21,5,6,12,0,1,bishop auckland,46.666667,11.111111,13.333333,26.666667,0.0,2.222222
6,5,0,0,0,0,0,"abram, wigan",100.0,0.0,0.0,0.0,0.0,0.0
7,31,7,29,4,9,6,wrexham,36.046512,8.139535,33.72093,4.651163,10.465116,6.976744
8,3,1,2,0,1,3,leven,30.0,10.0,20.0,0.0,10.0,30.0
9,8,9,11,0,5,18,bognor regis,15.686275,17.647059,21.568627,0.0,9.803922,35.294118


CPU times: user 1min 14s, sys: 60 ms, total: 1min 14s
Wall time: 1min 14s


### What is city-wise order status?
#### Takeaway: City "Manchester" has the highest pending (1.02%) order status. City "Stockport" has the highest processing (3.44%)  order status.

In [21]:
%%time
orderStatusByCity = pd.concat(
    list(map(Analyse.calIndDist,
            ["cityBilling"]*df.cityBilling.nunique(),
            df.cityBilling.unique(),
            ["orderStatus"]*df.cityBilling.nunique()))).dropna(how="all").fillna(0).reset_index(drop=True)
display(orderStatusByCity.head(15))

,completed,cityBilling,completed_%,pending payment,pending payment_%,processing,processing_%,on hold,on hold_%
0,48,bilston,100.0,0,0.0,0,0.0,0,0.0
1,113,rochester,100.0,0,0.0,0,0.0,0,0.0
2,964,manchester,98.973306,10,1.026694,0,0.0,0,0.0
3,27,inverness,100.0,0,0.0,0,0.0,0,0.0
4,1,ewloe,100.0,0,0.0,0,0.0,0,0.0
5,45,bishop auckland,100.0,0,0.0,0,0.0,0,0.0
6,5,"abram, wigan",100.0,0,0.0,0,0.0,0,0.0
7,86,wrexham,100.0,0,0.0,0,0.0,0,0.0
8,10,leven,100.0,0,0.0,0,0.0,0,0.0
9,51,bognor regis,100.0,0,0.0,0,0.0,0,0.0


CPU times: user 1min 12s, sys: 72 ms, total: 1min 12s
Wall time: 1min 12s


### What is city-wise payment method status?
#### Takeaway: Overall payment method ratio is in favour of debit or credit card. However, students from Leicester use paypal(49.4%) more than debit or credit card (47.4%). The same is also tru for city Bliston, Inverness, Bognor regis, Keith, and Northallerton. Students from Bishop auckland use paypal 24.4%  as their payment method.

In [22]:
%%time
paymentByCity = pd.concat(
    list(map(Analyse.calIndDist,
            ["cityBilling"]*df.cityBilling.nunique(),
            df.cityBilling.unique(),
            ["paymentMethodTitle"]*df.cityBilling.nunique()))).dropna(how="all").fillna(0).reset_index(drop=True)
display(paymentByCity.head(15))

,paypal,debit or credit card,apple pay (stripe),cityBilling,paypal_%,debit or credit card_%,apple pay (stripe)_%,chrome payment request (stripe),chrome payment request (stripe)_%,cash on delivery,cash on delivery_%
0,23,22,3,bilston,47.916667,45.833333,6.25,0,0.0,0,0.0
1,43,70,0,rochester,38.053097,61.946903,0.0,0,0.0,0,0.0
2,445,473,18,manchester,45.829042,48.712667,1.853759,35,3.604531,0,0.0
3,14,11,0,inverness,51.851852,40.740741,0.0,2,7.407407,0,0.0
4,0,1,0,ewloe,0.0,100.0,0.0,0,0.0,0,0.0
5,17,17,11,bishop auckland,37.777778,37.777778,24.444444,0,0.0,0,0.0
6,2,3,0,"abram, wigan",40.0,60.0,0.0,0,0.0,0,0.0
7,35,40,7,wrexham,40.697674,46.511628,8.139535,4,4.651163,0,0.0
8,4,5,1,leven,40.0,50.0,10.0,0,0.0,0,0.0
9,32,16,0,bognor regis,62.745098,31.372549,0.0,3,5.882353,0,0.0


CPU times: user 1min 12s, sys: 96.4 ms, total: 1min 12s
Wall time: 1min 12s


In [23]:
# Convert order date into pandas datetime format
df.orderDate = pd.to_datetime(df.orderDate)
df["date"] = df.orderDate.dt.date
df["day"] = df.orderDate.dt.day_name()
df["time"] = df.orderDate.dt.time
df["hour"] = df.orderDate.dt.hour
df["minute"] = df.orderDate.dt.minute
df["second"] = df.orderDate.dt.second

In [24]:
# Create decimal time combining hour and minute
df["time"] = df["hour"].astype("str") + "." + df["minute"].astype("str")
df.time = df.time.astype("float")

In [25]:
def createTimeQuarter(x):
    if 0<=x<=5.99:
        return "12_am_6_am"
    
    if 6<=x<=11.99:
        return "6_am_12_pm"
    
    if 12<=x<=17.99:
        return "12_pm_6_pm"
    
    if 18<=x<=23.99:
        return "6_pm_12_am"
    
    else:
        return "na"

# Apply the function and create time category
df["time"] = df.time.apply(createTimeQuarter)

### What is brand-wise time quarter distribution?
#### Takeaway: Overall most of the courses (35.4%) were purchased between 12 pm to 6 pm, followed by 25% for 6 pm to 12 am.
#### Best quarter for all brands is always 12 pm to 6 pm. 2nd best quarter for One Education is 6 pm to 12 am, while for TX its 6 am to 12 pm.

In [26]:
providerByTimeQuarter =  pd.concat(
    list(map(Analyse.calIndDist,
            ["courseProvider"]*df.courseProvider.nunique(),
            df.courseProvider.unique(),
            ["time"]*df.courseProvider.nunique()))).reset_index(drop=True)

overAllTimeQuarter = Analyse.calCombDist("time")
display(overAllTimeQuarter, providerByTimeQuarter)

,12_pm_6_pm,6_pm_12_am,6_am_12_pm,12_am_6_am,time,12_pm_6_pm_%,6_pm_12_am_%,6_am_12_pm_%,12_am_6_am_%
0,29836,21135,17384,16015,overall,35.363281,25.050373,20.60448,18.981866


,12_pm_6_pm,6_pm_12_am,6_am_12_pm,12_am_6_am,courseProvider,12_pm_6_pm_%,6_pm_12_am_%,6_am_12_pm_%,12_am_6_am_%
0,12494.0,9789.0,6718.0,2000,one education,40.301926,31.576401,21.670269,6.451405
1,NaN,NaN,NaN,11827,alpha academy,NaN,NaN,NaN,100.000000
2,4352.0,2994.0,2290.0,617,janets,42.446113,29.201209,22.334926,6.017751
3,3297.0,1823.0,2557.0,359,training express,41.027875,22.685416,31.819313,4.467397
4,4538.0,3148.0,2759.0,644,john academy,40.923438,28.388493,24.880512,5.807557
5,5155.0,3381.0,3060.0,568,course gate,42.379152,27.795133,25.156199,4.669517


### What is gender-wise time quarter distribution?
#### Takeaway: 65.4% students were female when the course were purchased between 12 am to 6 am.

In [27]:
timeQuarterByGender = pd.concat(
    list(map(Analyse.calIndDist,
            ["time"]*df.time.nunique(),
            df.time.unique(),
            ["gender"]*df.time.nunique()))).reset_index(drop=True)

display(timeQuarterByGender)

,F,M,time,F_%,M_%
0,13552,7583,6_pm_12_am,64.121126,35.878874
1,10776,6608,6_am_12_pm,61.988035,38.011965
2,10539,5476,12_am_6_am,65.807056,34.192944
3,19200,10636,12_pm_6_pm,64.351790,35.648210


### What is payment-method wise time quarter distribution?
#### Takeaway: Most of the payments methods were most used in between 12 pm to 6 pm.

In [28]:
timeQuarterByPaymentMethod = pd.concat(
    list(map(Analyse.calIndDist,
            ["paymentMethodTitle"]*df.paymentMethodTitle.nunique(),
            df.paymentMethodTitle.unique(),
            ["time"]*df.paymentMethodTitle.nunique()))).dropna(how="all").reset_index(drop=True)

display(timeQuarterByPaymentMethod)

,12_pm_6_pm,6_pm_12_am,6_am_12_pm,12_am_6_am,paymentMethodTitle,12_pm_6_pm_%,6_pm_12_am_%,6_am_12_pm_%,12_am_6_am_%
0,15174.0,10032.0,8773.0,7790.0,debit or credit card,36.328378,24.017812,21.003615,18.650195
1,12300.0,9457.0,6683.0,6654.0,paypal,35.048726,26.947626,19.043141,18.960506
2,820.0,644.0,350.0,230.0,apple pay (stripe),40.117417,31.506849,17.123288,11.252446
3,993.0,699.0,480.0,591.0,chrome payment request (stripe),35.939197,25.298588,17.372421,21.389794


#### What is order status-wise time quarter distribution?

In [29]:
timeQuarterByOrderStatus = pd.concat(
    list(map(Analyse.calIndDist,
            ["orderStatus"]*df.orderStatus.nunique(),
            df.orderStatus.unique(),
            ["time"]*df.orderStatus.nunique()))).dropna(how="all").reset_index(drop=True)

display(timeQuarterByOrderStatus)

,12_pm_6_pm,6_pm_12_am,6_am_12_pm,12_am_6_am,orderStatus,12_pm_6_pm_%,6_pm_12_am_%,6_am_12_pm_%,12_am_6_am_%
0,29589,20855,17186,15939,completed,35.406670,24.955426,20.565042,19.072862
1,125,149,99,40,pending payment,30.266344,36.077482,23.970944,9.685230
2,100,126,91,32,processing,28.653295,36.103152,26.074499,9.169054
3,22,5,8,4,on hold,56.410256,12.820513,20.512821,10.256410


#### What is brand-wise day distribution?

In [30]:
dayByProvider = pd.concat(
    list(map(Analyse.calIndDist,
            ["courseProvider"]*df.day.nunique(),
            df.courseProvider.unique(),
            ["day"]*df.day.nunique()))).reset_index(drop=True)
display(dayByProvider)

,Friday,Monday,Thursday,Tuesday,Wednesday,Saturday,Sunday,courseProvider,Friday_%,Monday_%,Thursday_%,Tuesday_%,Wednesday_%,Saturday_%,Sunday_%
0,5202,5196,4812,4761,4692,3240,3098,one education,16.780104,16.760750,15.522080,15.357569,15.134996,10.451276,9.993226
1,1611,1858,1757,1843,1716,1541,1501,alpha academy,13.621375,15.709817,14.855838,15.582988,14.509174,13.029509,12.691300
2,1649,1555,1328,1585,1474,1226,1436,janets,16.083098,15.166293,12.952307,15.458890,14.376280,11.957476,14.005657
3,1329,1284,1455,1300,1195,718,755,training express,16.538079,15.978099,18.106023,16.177203,14.870582,8.934793,9.395222
4,1582,1767,1730,1707,1713,1248,1342,john academy,14.266390,15.934710,15.601046,15.393633,15.447741,11.254396,12.102083
5,1925,2072,1904,2262,1878,1141,982,course gate,15.825386,17.033870,15.652746,18.595857,15.439000,9.380138,8.073002


#### What is payment method-wise day distribution?

In [31]:
dayByPaymentMethod = pd.concat(
    list(map(Analyse.calIndDist,
            ["paymentMethodTitle"]*df.day.nunique(),
            df.paymentMethodTitle.unique(),
            ["day"]*df.day.nunique()))).reset_index(drop=True).dropna(how="all").fillna(0)
display(dayByPaymentMethod)

,Monday,Tuesday,Friday,Thursday,Wednesday,Sunday,Saturday,paymentMethodTitle,Monday_%,Tuesday_%,Friday_%,Thursday_%,Wednesday_%,Sunday_%,Saturday_%
0,6819.0,6774.0,6583.0,6414.0,6240.0,4528.0,4411.0,debit or credit card,16.325505,16.217769,15.760492,15.355886,14.939309,10.840576,10.560464
1,5739.0,5483.0,5457.0,5270.0,5274.0,3866.0,4005.0,paypal,16.353223,15.623753,15.549667,15.016812,15.028210,11.016128,11.412207
2,348.0,306.0,332.0,309.0,303.0,236.0,210.0,apple pay (stripe),17.025440,14.970646,16.242661,15.117417,14.823875,11.545988,10.273973
3,426.0,456.0,453.0,442.0,420.0,273.0,293.0,chrome payment request (stripe),15.418024,16.503800,16.395223,15.997105,15.200869,9.880565,10.604415
5,0.0,3.0,0.0,0.0,0.0,0.0,0.0,cash on delivery,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000


####  What is order status-wise day distribution?

In [32]:
dayByOrderStatus = pd.concat(
    list(map(Analyse.calIndDist,
            ["orderStatus"]*df.day.nunique(),
            df.orderStatus.unique(),
            ["day"]*df.day.nunique()))).reset_index(drop=True).dropna(how="all").fillna(0)
display(dayByOrderStatus)

,Monday,Tuesday,Friday,Thursday,Wednesday,Sunday,Saturday,orderStatus,Monday_%,Tuesday_%,Friday_%,Thursday_%,Wednesday_%,Sunday_%,Saturday_%
0,13601,13323,13168,12870,12556,9052,8999,completed,16.275174,15.942515,15.757039,15.400448,15.024710,10.831768,10.768347
1,77,76,60,53,51,27,69,pending payment,18.644068,18.401937,14.527845,12.832930,12.348668,6.537530,16.707022
2,52,51,58,59,53,31,45,processing,14.899713,14.613181,16.618911,16.905444,15.186246,8.882521,12.893983
3,2,8,12,4,8,4,1,on hold,5.128205,20.512821,30.769231,10.256410,20.512821,10.256410,2.564103


#### What is gender-wise day distribution?

In [33]:
dayByGender = pd.concat(
    list(map(Analyse.calIndDist,
            ["gender"]*df.day.nunique(),
            df.gender.unique(),
            ["day"]*df.day.nunique()))).reset_index(drop=True).dropna(how="all").fillna(0)
display(dayByGender)

,Monday,Thursday,Friday,Tuesday,Wednesday,Saturday,Sunday,gender,Monday_%,Thursday_%,Friday_%,Tuesday_%,Wednesday_%,Saturday_%,Sunday_%
0,5037,4929,4818,4687,4330,3285,3217,M,16.622117,16.265716,15.899416,15.467115,14.289014,10.840511,10.616111
1,8695,8057,8480,8771,8338,5829,5897,F,16.081898,14.901881,15.684244,16.222465,15.421607,10.781068,10.906838
